In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure
from scipy import ndimage, signal
from flowdec import data as fd_data
from flowdec import psf as fd_psf
from flowdec import restoration as fd_restoration

In [ ]:
pip install DBUtils

In [ ]:
actual = fd_data.neuron_25pct().data
actual.shape

In [ ]:
kernel = np.zeros_like(actual)
for offset in [0, 1]:
    kernel[tuple((np.array(kernel.shape) - offset) // 2)] = 1
kernel = ndimage.gaussian_filter(kernel, sigma=1.)
kernel.shape

In [ ]:
data = signal.fftconvolve(actual, kernel, mode='same')
data.shape

In [ ]:
algo = fd_restoration.RichardsonLucyDeconvolver(data.ndim, pad_min=np.ones(data.ndim)).initialize()
res = algo.run(fd_data.Acquisition(data=data, kernel=kernel), niter=5)

In [ ]:
%%time
# Note that deconvolution initialization is best kept separate from execution since the "initialize" 
# operation corresponds to creating a TensorFlow graph, which is a relatively expensive operation and
# should not be repeated across iterations if deconvolving more than one image
algo = fd_restoration.RichardsonLucyDeconvolver(data.ndim).initialize()
res = algo.run(fd_data.Acquisition(data=data, kernel=kernel), niter=100).data

In [ ]:
fig, axs = plt.subplots(1, 3)
axs = axs.ravel()
fig.set_size_inches(18, 12)
center = tuple([slice(None), slice(10, -10), slice(10, -10)])
titles = ['Original Image', 'Blurred Image', 'Reconstructed Image']
for i, d in enumerate([actual, data, res]):
    img = exposure.adjust_gamma(d[center].max(axis=0), gamma=.2)
    axs[i].imshow(img, cmap='Spectral_r')
    axs[i].set_title(titles[i])
    axs[i].axis('off')

#PART 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio

In [ ]:
def uniform_noise(size, prob=0.1):
    '''
    Generates a matrix with uniform noise in the range [0-255] to be added to an image
    
    :param size: tuple defining the size of the noise matrix 
    :param prob: probability for the uniform noise generation 
    :type prob: float
    :return matrix with uniform noise to be added to image
    '''
    
    levels = int((prob * 255) // 2)
    noise = np.random.randint(-levels, levels, size)
    
    return noise

In [ ]:
img = imageio.imread("0.jpg")
np.unique(img)

In [ ]:
uni_noise = uniform_noise(img.shape, prob=0.1)
img_uni = np.clip(img.astype(int)+uni_noise, 0, 255)

hist_img,_ = np.histogram(img, bins=256, range=(0,255))
hist_uni,_ = np.histogram(img_uni, bins=256, range=(0,255))

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(121)
plt.imshow(img, cmap="gray", vmin=0, vmax=255)
plt.axis('off')
plt.subplot(122)
plt.bar(np.arange(256), hist_img)

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(121)
plt.imshow(img_uni, cmap="gray", vmin=0, vmax=255)
plt.axis('off')
plt.subplot(122)
plt.bar(np.arange(256), hist_uni)

#print(np.unique(img_uni))

In [ ]:
def gaussian_noise(size, mean=0, std=0.01):
    '''
    Generates a matrix with Gaussian noise in the range [0-255] to be added to an image
    
    :param size: tuple defining the size of the noise matrix 
    :param mean: mean of the Gaussian distribution
    :param std: standard deviation of the Gaussian distribution, default 0.01
    :return matrix with Gaussian noise to be added to image
    '''
    noise = np.multiply(np.random.normal(mean, std, size), 255)
    
    return noise

In [ ]:
# creating the noise matrix to be added
gau_noise = gaussian_noise(img.shape, mean=0, std=0.05)

# adding and clipping values below 0 or above 255
img_gau = np.clip(img.astype(int)+gau_noise, 0, 255)

hist_gau,_ = np.histogram(img_gau, bins=256, range=(0,255))

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(121)
plt.imshow(img_gau, cmap="gray", vmin=0, vmax=255)
plt.axis('off')
plt.subplot(122)
plt.bar(np.arange(256), hist_gau)

In [ ]:
def impulsive_noise(image, prob=0.1, mode='salt_and_pepper'):
    '''
    Returns image with impulsive noise (0 and/or 255) to replace pixels in the image with some probability
    
    :param image: input image
    :param prob: probability for the impulsive noise generation 
    :param mode: type of noise, 'salt', 'pepper' or 'salt_and_pepper' (default)
    :type prob: float
    :return noisy image with impulsive noise
    '''

    noise = np.array(image, copy=True)
    for x in np.arange(image.shape[0]):
        for y in np.arange(image.shape[1]):
            rnd = np.random.random()
            if rnd < prob:
                rnd = np.random.random()
                if rnd > 0.5:
                    noise[x,y] = 255
                else:
                    noise[x,y] = 0
    
    return noise

In [ ]:
img_imp = impulsive_noise(img, prob=0.1)

hist_imp,_ = np.histogram(img_imp, bins=256, range=(0,255))

In [ ]:

plt.figure(figsize=(14,6))
plt.subplot(121)
plt.imshow(img_imp, cmap="gray", vmin=0, vmax=255)
plt.axis('off')
plt.subplot(122)
plt.bar(np.arange(256), hist_imp)

In [ ]:

# generating noise and subtracting it from the image
uni_noise_new = uniform_noise(img.shape, prob=0.1)
img_uni_res = img_uni - uni_noise_new

def rmse(f,g):
    size = f.shape
    return np.sqrt(np.sum(np.square(f-g))/(size[0]*size[1]))

print("Error between noisy and original: %.3f%%" % (rmse(img.astype(float), img_uni.astype(float))))
print("Error between 'restored' and original: %.3f%%" % (rmse(img_uni_res.astype(float), img.astype(float))))

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(131)
plt.imshow(img, cmap="gray", vmin=0, vmax=255)
plt.axis('off')
plt.subplot(132)
plt.imshow(img_uni, cmap="gray", vmin=0, vmax=255)
plt.axis('off')
plt.subplot(133)
plt.imshow(img_uni_res, cmap="gray", vmin=0, vmax=255)
plt.axis('off')

In [ ]:
# importing the fft algorithms
# including the transform, inverse and the shift methods
from scipy.fftpack import fftn, ifftn, fftshift

In [ ]:
img_imp = impulsive_noise(img, prob=0.05)

w_mean3 = np.ones([3,3])/float(3*3)
img_mean = fft_imagefilter(img_imp, w_mean3)
img_median = medianfilter(img_imp, 3)

#PART 3

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

import numpy as np
import numexpr as ne
from scipy.ndimage import correlate1d
from dphutils import scale
import scipy.signal
from timeit import Timer

In [ ]:
pip install DBUtils

In [ ]:
x = np.arange(-2,2,0.05)

# parameter sigma is related to the dispersion of the values
sigma1 = 0.3
gauss1d_1 = (1/(sigma1*np.sqrt(2*np.pi))) * np.exp(-(np.square(x/sigma1)/2))

sigma2 = 0.1
gauss1d_2 = (1/(sigma2*np.sqrt(2*np.pi))) * np.exp(-(np.square(x/sigma2)/2))

plt.plot(x, gauss1d_1, 'r') # larger sigma, 0.3
plt.plot(x, gauss1d_2, 'b') # smaller sigma, 0.1

In [ ]:
def gaussian_filter(k=5, sigma=1.0):
    ''' Gaussian filter
    :param k: defines the lateral size of the kernel/filter, default 5
    :param sigma: standard deviation (dispersion) of the Gaussian distribution
    :return matrix with a filter [k x k] to be used in convolution operations
    '''
    arx = np.arange((-k // 2) + 1.0, (k // 2) + 1.0)
    x, y = np.meshgrid(arx, arx)
    filt = np.exp(-(1/2) * (np.square(x) + np.square(y)) / np.square(sigma))
    return filt / np.sum(filt)

In [ ]:
g1 = gaussian_filter(k=5, sigma=0.9)
print(g1)

In [ ]:
g2 = gaussian_filter(k=9, sigma=1.5)
plt.imshow(g2, cmap='hot', interpolation='nearest')
plt.colorbar()

In [ ]:

f = imageio.imread("0.jpg")
h = gaussian_filter(k=7, sigma=2.5)

# computing the number of padding on one side
a = int(f.shape[0]//2 - h.shape[0]//2)
h_pad = np.pad(h, (a,a-1), 'constant', constant_values=(0))

# computing the Fourier transforms
F = fftn(f)
H = fftn(h_pad)

plt.subplot(121)
plt.imshow(fftshift(np.log(np.abs(F)+1)), cmap="gray")
plt.subplot(122)
plt.imshow(fftshift(np.log(np.abs(H)+1)), cmap="gray")

# convolution
G = np.multiply(F,H)

# Inverse Transform
# - we have to perform FFT shift before reconstructing the image in the space domain
g = fftshift(ifftn(G).real)

plt.figure(figsize=(12,5))
plt.subplot(121)
plt.imshow(f, cmap="gray", vmin=0, vmax=128); plt.title("original image")
plt.subplot(122)
plt.imshow(g, cmap="gray", vmin=0, vmax=128); plt.title("degraded/blurred image")

Part-2

In [ ]:
# imports for the model
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2DTranspose, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50

# imports for the dataset
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
#os.chdir('D:/Traffic_Sign_Recognition')
from sklearn.model_selection import train_test_split
#from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator,DirectoryIterator
import tensorflow_addons as tfa

In [ ]:
data = []
labels = []
classes = 5
cur_path = 'data1'

In [ ]:
dir_ = os.path.join('data1')

In [ ]:
train_datagen = ImageDataGenerator(validation_split = 0.2)
#test_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(dir_,target_size=(32,32),
                                                   batch_size= 1920,
                                                   class_mode='categorical',
                                                   shuffle=True,
                                                   subset = 'training')

test_generator = train_datagen.flow_from_directory(dir_,
                                                          target_size = (32,32),
                                                          batch_size = 480,
                                                          class_mode = 'categorical',
                                                          shuffle=True,
                                                          subset = 'validation')

In [ ]:
x_train, y_train = next(train_generator)
x_test, y_test  = next(test_generator)

In [ ]:
for i in range(classes):
    path = os.path.join(cur_path,str(i))
    print(path)
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '/'+ a)
            image = image.resize((32,32))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except Exception as e:
            print(e)

In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=0)
print(x_train.shape)

In [ ]:
# x_train = (x_train / 255.0).astype(np.float32)
# x_test  = (x_test  / 255.0).astype(np.float32)

# y_train = to_categorical(y_train)
# y_test  = to_categorical(y_test)

In [ ]:
# Get a pre-built ResNet50 w/o the top layer (classifer) and input shape configured for 128 x 128
base = ResNet50(include_top=False, input_shape=(128, 128, 3), pooling='max')

# Add a new classifier (top) layer for the 10 classes in CIFAR-10
outputs = Dense(36, activation='softmax')(base.output)

# Rebuild the model with the new classifier
resnet = Model(base.input, outputs)
#resnet.summary()

In [ ]:
# Create the pre-stem as a Sequential model
model = Sequential()



# Add the ResNet50 model as the remaining layers and rebuild
model.add(resnet)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['acc'])

#model.summary()

In [ ]:
history=model.fit(x_train, y_train, epochs=20, batch_size=16, validation_split=0.2)

In [ ]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=[6,5])
plt.plot(history.history['acc'],'C1',linewidth=3.0)
plt.plot(history.history['val_acc'],'C0',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=10)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [ ]:
plt.figure(figsize=[6,5])
plt.plot(history.history['loss'],'C1',linewidth=3.0)
plt.plot(history.history['val_loss'],'C0',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=10)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)

In [ ]:
y_test_arg=np.argmax(y_test,axis=1)
Y_pred = np.argmax(model.predict(x_test),axis=1)

In [ ]:
import seaborn as sns
from sklearn.metrics import multilabel_confusion_matrix, classification_report, confusion_matrix
cm=confusion_matrix(y_test_arg, Y_pred)
fig, ax = plt.subplots(figsize=(10,10))
ax = sns.heatmap(cm, annot=True, cmap='BuGn', ax=ax)

ax.set_title('Confusion Matrix for Sign Language Recognition \n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');
#ax.xaxis.set_ticklabels(['0', '1', '2']); ax.yaxis.set_ticklabels(['0', '1', '2']);
plt.show()